<a href="https://colab.research.google.com/github/lahirunie-dulsara/EN3150-Assignment-3-CNN/blob/Lahirunie/appendicities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ucimlrepo

In [11]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
regensburg_pediatric_appendicitis = fetch_ucirepo(id=938)

# data (as pandas dataframes)
X = regensburg_pediatric_appendicitis.data.features
y = regensburg_pediatric_appendicitis.data.targets
z = regensburg_pediatric_appendicitis.data.other

# metadata
print(regensburg_pediatric_appendicitis.metadata)

# variable information
print(regensburg_pediatric_appendicitis.variables)

{'uci_id': 938, 'name': 'Regensburg Pediatric Appendicitis', 'repository_url': 'https://archive.ics.uci.edu/dataset/938/regensburg+pediatric+appendicitis', 'data_url': 'https://archive.ics.uci.edu/static/public/938/data.csv', 'abstract': 'This repository holds the data from a cohort of pediatric patients with suspected appendicitis admitted with abdominal pain to Children’s Hospital St. Hedwig in Regensburg, Germany, between 2016 and 2021. Each patient has (potentially multiple) ultrasound (US) images, aka views, tabular data comprising laboratory, physical examination, scoring results and ultrasonographic findings extracted manually by the experts, and three target variables, namely, diagnosis, management and severity.', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Tabular', 'Image'], 'num_instances': 782, 'num_features': 53, 'feature_types': ['Real', 'Categorical', 'Integer'], 'demographics': ['Age', 'Sex'], 'target_col': ['Management', 'Severity',

In [3]:
print(X)

       Age    BMI     Sex  Height  Weight  Length_of_Stay  Alvarado_Score  \
0    12.68  16.90  female   148.0    37.0             3.0             4.0   
1    14.10  31.90    male   147.0    69.5             2.0             5.0   
2    14.14  23.30  female   163.0    62.0             4.0             5.0   
3    16.37  20.60  female   165.0    56.0             3.0             7.0   
4    11.08  16.90  female   163.0    45.0             3.0             5.0   
..     ...    ...     ...     ...     ...             ...             ...   
777  12.41  25.25  female   166.5    70.0             4.0             8.0   
778  17.09  20.43  female   158.0    51.0             6.0             5.0   
779  14.99  19.91  female   152.0    46.0             4.0             5.0   
780   7.20  14.30    male   129.3    23.9             5.0             9.0   
781  11.51  18.17    male   146.5    39.0             4.0             2.0   

     Paedriatic_Appendicitis_Score Appendix_on_US  Appendix_Diameter  ...  

In [4]:
print(y)

             Management       Severity        Diagnosis
0          conservative  uncomplicated     appendicitis
1          conservative  uncomplicated  no appendicitis
2          conservative  uncomplicated  no appendicitis
3          conservative  uncomplicated  no appendicitis
4          conservative  uncomplicated     appendicitis
..                  ...            ...              ...
777    primary surgical  uncomplicated     appendicitis
778  secondary surgical    complicated     appendicitis
779    primary surgical  uncomplicated     appendicitis
780    primary surgical  uncomplicated     appendicitis
781    primary surgical  uncomplicated     appendicitis

[782 rows x 3 columns]


In [13]:
print(regensburg_pediatric_appendicitis)

{'data': {'ids': None, 'features':        Age    BMI     Sex  Height  Weight  Length_of_Stay  Alvarado_Score  \
0    12.68  16.90  female   148.0    37.0             3.0             4.0   
1    14.10  31.90    male   147.0    69.5             2.0             5.0   
2    14.14  23.30  female   163.0    62.0             4.0             5.0   
3    16.37  20.60  female   165.0    56.0             3.0             7.0   
4    11.08  16.90  female   163.0    45.0             3.0             5.0   
..     ...    ...     ...     ...     ...             ...             ...   
777  12.41  25.25  female   166.5    70.0             4.0             8.0   
778  17.09  20.43  female   158.0    51.0             6.0             5.0   
779  14.99  19.91  female   152.0    46.0             4.0             5.0   
780   7.20  14.30    male   129.3    23.9             5.0             9.0   
781  11.51  18.17    male   146.5    39.0             4.0             2.0   

     Paedriatic_Appendicitis_Score Appen

In [12]:
print(z)

None


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import zipfile

In [7]:
# Path to your zip file in Drive
zip_path = '/content/drive/MyDrive/CNN_Assignment3/US_Pictures.zip'

# Extract the zip file to a local folder in Colab
extract_dir = '/content/US_Pictures'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Dataset extracted to:", extract_dir)

Dataset extracted to: /content/US_Pictures


In [8]:
import os
import re
from PIL import Image

dataset_dir = extract_dir  # path where the dataset was extracted

selected_images = []
selected_filenames = []

# Regular expression: match 'app' or 'appendix' as the only word (after ignoring leading numbers)
pattern = re.compile(r'^(?:[\d\.\s_-]*)(app|appendix)$', re.IGNORECASE)

# Allowed file extensions
allowed_exts = ('.bmp', '.png')

for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.lower().endswith(allowed_exts):
            # Remove file extension
            name_without_ext = os.path.splitext(file)[0]
            # Remove leading/trailing spaces
            cleaned_name = name_without_ext.strip()

            # Match only if the cleaned name fits the pattern
            if pattern.match(cleaned_name):
                img_path = os.path.join(root, file)
                img = Image.open(img_path).convert('RGB')
                selected_images.append(img)
                selected_filenames.append(file)

print(f"Total selected images: {len(selected_images)}")


Total selected images: 679


In [9]:
print(selected_images)
print(selected_filenames)

[<PIL.Image.Image image mode=RGB size=716x537 at 0x7DA58CA98440>, <PIL.Image.Image image mode=RGB size=716x537 at 0x7DA58C791040>, <PIL.Image.Image image mode=RGB size=716x537 at 0x7DA5A18AC7D0>, <PIL.Image.Image image mode=RGB size=793x577 at 0x7DA5A1871670>, <PIL.Image.Image image mode=RGB size=792x580 at 0x7DA587DE4CE0>, <PIL.Image.Image image mode=RGB size=716x537 at 0x7DA587DE4D70>, <PIL.Image.Image image mode=RGB size=716x537 at 0x7DA587DE4DD0>, <PIL.Image.Image image mode=RGB size=716x537 at 0x7DA587DE4E30>, <PIL.Image.Image image mode=RGB size=1268x921 at 0x7DA587DE4E90>, <PIL.Image.Image image mode=RGB size=716x537 at 0x7DA5A08088C0>, <PIL.Image.Image image mode=RGB size=716x537 at 0x7DA587DE4F50>, <PIL.Image.Image image mode=RGB size=800x585 at 0x7DA587DE4EF0>, <PIL.Image.Image image mode=RGB size=799x587 at 0x7DA587DE5040>, <PIL.Image.Image image mode=RGB size=716x521 at 0x7DA587DE5070>, <PIL.Image.Image image mode=RGB size=716x537 at 0x7DA587DE5100>, <PIL.Image.Image image 

In [10]:
import os

# Path to the folder where you want to save selected images
save_dir = '/content/Selected_Images'

# Create the folder if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Loop through selected images and save them
for img, filename in zip(selected_images, selected_filenames):
    save_path = os.path.join(save_dir, filename)
    img.save(save_path)  # PIL will save in the original format automatically

print(f"Saved {len(selected_images)} images to {save_dir}")


Saved 679 images to /content/Selected_Images


In [17]:
import pandas as pd

# Replace with your actual path
file_path = '/content/drive/MyDrive/CNN_Assignment3/app_data.xlsx'

df = pd.read_excel(file_path)
print(df.head())


     Age   BMI     Sex  Height  Weight  Length_of_Stay    Management  \
0  12.68  16.9  female   148.0    37.0             3.0  conservative   
1  14.10  31.9    male   147.0    69.5             2.0  conservative   
2  14.14  23.3  female   163.0    62.0             4.0  conservative   
3  16.37  20.6  female   165.0    56.0             3.0  conservative   
4  11.08  16.9  female   163.0    45.0             3.0  conservative   

        Severity Diagnosis_Presumptive        Diagnosis  ...  \
0  uncomplicated          appendicitis     appendicitis  ...   
1  uncomplicated          appendicitis  no appendicitis  ...   
2  uncomplicated          appendicitis  no appendicitis  ...   
3  uncomplicated          appendicitis  no appendicitis  ...   
4  uncomplicated          appendicitis     appendicitis  ...   

   Abscess_Location  Pathological_Lymph_Nodes Lymph_Nodes_Location  \
0               NaN                       yes                 reUB   
1               NaN                       

In [23]:
import pandas as pd
import re


# Check the first few rows to confirm column names
# print("Columns:", app_data.columns)
# print(app_data.head())

# === Step 2: Suppose your selected image filenames are stored in a list ===
# Example list (replace this with your actual list)

# === Step 3: Extract US_Number from each image name (integer before decimal) ===
image_data = []
for img in selected_filenames:
    match = re.match(r"(\d+)", img)
    if match:
        us_number = int(match.group(1))
        image_data.append({"Image Name": img, "US_Number": us_number})

image_df = pd.DataFrame(image_data)

# === Step 4: Merge with diagnosis data ===
# Ensure column names match exactly
if 'US_Number' not in app_data.columns or 'Diagnosis' not in app_data.columns:
    raise ValueError("app_data.xlsx must contain 'US_Number' and 'Diagnosis' columns")

merged_df = pd.merge(image_df, app_data[['US_Number', 'Diagnosis']], on='US_Number', how='left')

# === Step 5: Display final table ===
print("\n Final Table:")
print(merged_df)

# === (Optional) Save to Excel/CSV ===
merged_df.to_excel('/content/selected_images_with_diagnosis.xlsx', index=False)
print("\nSaved as 'selected_images_with_diagnosis.xlsx'")



 Final Table:
         Image Name  US_Number     Diagnosis
0     887.3 App.bmp        887  appendicitis
1    96.12 App .bmp         96  appendicitis
2    747.10 App.bmp        747           NaN
3     152.2 App.bmp        152  appendicitis
4     918.2 App.bmp        918  appendicitis
..              ...        ...           ...
674   169.5 App.bmp        169  appendicitis
675   550.1 App.bmp        550  appendicitis
676   838.1 App.bmp        838           NaN
677  102.12 App.bmp        102           NaN
678   841.5 App.bmp        841           NaN

[679 rows x 3 columns]

Saved as 'selected_images_with_diagnosis.xlsx'


In [29]:
# Filter and print rows with diagnosis 'no appendicitis'
no_appendicitis_df = merged_df[merged_df['Diagnosis'].str.lower() == 'no appendicitis']
print(no_appendicitis_df)


            Image Name  US_Number        Diagnosis
22    7.1 Appendix.bmp          7  no appendicitis
26       278.2 App.bmp        278  no appendicitis
44       233.5 App.bmp        233  no appendicitis
46       583.4 App.bmp        583  no appendicitis
59      547.11 App.bmp        547  no appendicitis
..                 ...        ...              ...
625      711.2 App.bmp        711  no appendicitis
630      266.2 App.bmp        266  no appendicitis
633   5.3 Appendix.bmp          5  no appendicitis
654      348.1 App.bmp        348  no appendicitis
667  25.1 Appendix.bmp         25  no appendicitis

[79 rows x 3 columns]


In [30]:
# Drop rows with NaN in the Diagnosis column
cleaned_df = merged_df.dropna(subset=['Diagnosis'])

# Print the new table
print(cleaned_df)

# (Optional) Save the cleaned table as a new CSV file
cleaned_df.to_csv('cleaned_labeled_images.csv', index=False)


         Image Name  US_Number     Diagnosis
0     887.3 App.bmp        887  appendicitis
1    96.12 App .bmp         96  appendicitis
3     152.2 App.bmp        152  appendicitis
4     918.2 App.bmp        918  appendicitis
5     130.6 App.bmp        130  appendicitis
..              ...        ...           ...
669   781.2 App.bmp        781  appendicitis
672   457.2 App.bmp        457  appendicitis
673    43.2 App.bmp         43  appendicitis
674   169.5 App.bmp        169  appendicitis
675   550.1 App.bmp        550  appendicitis

[572 rows x 3 columns]
